<a href="https://colab.research.google.com/github/vikasvinay/ML/blob/tensorflow-flower/tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numpy


In [11]:
import tensorflow as tf
import os

**Preprocessing Data** 

In [12]:
_url = 'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz'
zip_file = tf.keras.utils.get_file(origin=_url, fname='flower_photos.tgz',extract=True, cache_subdir='/')
base_dir = os.path.join(os.path.dirname(zip_file), 'flower_photos')

In [13]:
IMAGE_SIZE = 224
BATCH_SIZE = 64
dataGen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)
train_generater = dataGen.flow_from_directory(
    base_dir, 
    target_size=(IMAGE_SIZE,IMAGE_SIZE), 
    batch_size=BATCH_SIZE, 
    subset='training')

val_genetater = dataGen.flow_from_directory(
    base_dir, 
    target_size=(IMAGE_SIZE,IMAGE_SIZE), 
    batch_size=BATCH_SIZE, 
    subset='validation')

Found 2939 images belonging to 5 classes.
Found 731 images belonging to 5 classes.


**Building Nural network**


In [14]:
print(train_generater.class_indices)

{'daisy': 0, 'dandelion': 1, 'roses': 2, 'sunflowers': 3, 'tulips': 4}


In [18]:

lables = '\n'.join(sorted(train_generater.class_indices.keys()))
with open('lables.txt','w') as f:
  f.write(lables)

In [19]:
IMAGE_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

base_model = tf.keras.applications.MobileNetV2(input_shape=IMAGE_SHAPE, include_top=False, weights='imagenet')

9412608/9406464 [==============================] - 0s 0us/step


In [20]:
base_model.trainable = False

In [23]:
model = tf.keras.Sequential([
                             base_model,
                             tf.keras.layers.Conv2D(32,3, activation='relu'),
                             tf.keras.layers.Dropout(0.2),
                             tf.keras.layers.GlobalAveragePooling2D(),
                             tf.keras.layers.Dense(5, activation= 'softmax')
])

In [24]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'], )

In [25]:
epochs = 10
history = model.fit(train_generater,
                    epochs=epochs,
                    validation_data=val_genetater)

Epoch 1/10
46/46 [==============================] - 137s 3s/step - loss: 1.3998 - accuracy: 0.5562 - val_loss: 0.4774 - val_accuracy: 0.8276
Epoch 2/10
46/46 [==============================] - 132s 3s/step - loss: 0.3303 - accuracy: 0.8853 - val_loss: 0.4398 - val_accuracy: 0.8399
Epoch 3/10
46/46 [==============================] - 131s 3s/step - loss: 0.2076 - accuracy: 0.9261 - val_loss: 0.4005 - val_accuracy: 0.8591
Epoch 4/10
46/46 [==============================] - 131s 3s/step - loss: 0.1187 - accuracy: 0.9626 - val_loss: 0.4541 - val_accuracy: 0.8564
Epoch 5/10
46/46 [==============================] - 131s 3s/step - loss: 0.1237 - accuracy: 0.9575 - val_loss: 0.4061 - val_accuracy: 0.8728
Epoch 6/10
46/46 [==============================] - 131s 3s/step - loss: 0.0576 - accuracy: 0.9848 - val_loss: 0.4318 - val_accuracy: 0.8796
Epoch 7/10
46/46 [==============================] - 131s 3s/step - loss: 0.0391 - accuracy: 0.9931 - val_loss: 0.4581 - val_accuracy: 0.8605
Epoch 8/10
46

**Saving as tflite**

In [27]:
saved_model_dir = ''
tf.saved_model.save(model, saved_model_dir)
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
  f.write(tflite_model)


INFO:tensorflow:Assets written to: assets


INFO:tensorflow:Assets written to: assets


**Download the model**

In [29]:
from google.colab import files
files.download('model.tflite')
# files.download('lables.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>